In [1]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Input, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score

In [2]:
# Import the data
batchsize = 8
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)

In [3]:

train_data = train_datagen.flow_from_directory(r'C:\Users\INFINIX\Driver Drowsiness Detection System\Prepared_Data\train', target_size=(299, 299),
                                               batch_size=batchsize, class_mode='categorical', subset='training')

validation_data = train_datagen.flow_from_directory(r'C:\Users\INFINIX\Driver Drowsiness Detection System\Prepared_Data\train', target_size=(299, 299),
                                               batch_size=batchsize, class_mode='categorical', subset='validation')


Found 64105 images belonging to 2 classes.
Found 16025 images belonging to 2 classes.


In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(
    r'C:\Users\INFINIX\Driver Drowsiness Detection System\Prepared_Data\test',
    target_size=(299, 299),  # Xception model input shape
    batch_size=batchsize,
    class_mode='categorical'
)

Found 4768 images belonging to 2 classes.


In [5]:
# Model training
bmodel = Xception(include_top=False, weights='imagenet', input_tensor=Input(shape=(299, 299, 3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs=hmodel)

for layer in bmodel.layers:
    layer.trainable = False

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 149, 149, 32)         864       ['input_1[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 149, 149, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 149, 149, 32)         0         ['block1_conv1_bn[0][0]'] 

In [7]:
checkpoint = ModelCheckpoint(
    'path_to_model',
    monitor='val_loss',
    save_best_only=True,
    verbose=3
)

earlystop = EarlyStopping(
    monitor='val_loss',
    patience=7,
    verbose=3,
    restore_best_weights=True
)

learning_rate = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    verbose=3,
)

callbacks = [checkpoint, earlystop, learning_rate]

In [8]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(
    train_data,
    steps_per_epoch=train_data.samples//batchsize,
    validation_data=validation_data,
    validation_steps=validation_data.samples//batchsize,
    callbacks=callbacks,
    epochs=5
)

C:\Users\INFINIX\AppData\Local\Temp\ipykernel_41452\2635639900.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/5


8013/8013 [==============================] - ETA: 0s - loss: 0.2402 - accuracy: 0.9089


KeyboardInterrupt



In [9]:
# Evaluate the model
acc_test, loss_test = model.evaluate(test_data)
print("Test Accuracy:", acc_test)
print("Test Loss:", loss_test)


596/596 [==============================] - 837s 1s/step - loss: 0.2803 - accuracy: 0.9054
Test Accuracy: 0.28025904297828674
Test Loss: 0.9054110646247864


In [ ]:
acc_vr, loss_vr = model.evaluate(validation_data)
print("Validation Accuracy:", acc_vr)
print("Validation Loss:", loss_vr)

 218/2004 [==>...........................] - ETA: 44:57 - loss: 0.1353 - accuracy: 0.9541

In [ ]:
acc_tr, loss_tr = model.evaluate(train_data)
print("Training Accuracy:", acc_tr)
print("Training Loss:", loss_tr)